In [1]:
%cd ..

/Users/Matteo/PycharmProjects/TrafficEmu


# Bayesian optimization: finding the minimum emission

In this notebook we will performing bayesian optimization on the emulator trained on our sumo simulation. We are interested in identifying the parameters which enable us to obtain the minimum travel time. Our emulator is a gaussian process with an **rbf** kernel.


## Imports

In [2]:
import pickle
import numpy as np
import emukit as ek
import GPy
import pandas as pd

from emukit.model_wrappers import GPyModelWrapper
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from experimental_design import config
from sumo_grid_simulation.grid_simulation import Simulator

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


## Simulator

We start by initialising our simulation.

In [3]:
simulator = Simulator(end_time=300)

We then define the two user functions in which we are interested:

In [4]:
def user_function_emissions(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function emissions called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )

        CO2 = s['CO2']/s['num_emissions_samples']
        result.append(CO2/s['duration'])
        print(f'\nOutput {result[-1]}\n')
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

We then load the parameters' space from our configuration file:

In [5]:
parameter_space = config.get_parameter_space()

## Emulator

### Rnadomly initialised emulator

The first emulator we are going to analyse is the one that is just initilised with random points

We first sample at random 200 datapoints from the parameter space:

In [6]:
init_X, init_Y = pickle.load(open('bayesian_optimization/init_points/270_init_points_emissions.pkl', 'rb'))

'''
design = RandomDesign(parameter_space)
num_init_points = 270
init_X = design.get_samples(num_init_points)
init_Y = user_function_emissions(init_X)
init_points = init_X, init_Y
with open(f'bayesian_optimization/init_points/{num_init_points}_init_points_emissions.pkl', "wb") as f:
     pickle.dump(init_points, f)
'''

'\ndesign = RandomDesign(parameter_space)\nnum_init_points = 270\ninit_X = design.get_samples(num_init_points)\ninit_Y = user_function_emissions(init_X)\ninit_points = init_X, init_Y\nwith open(f\'bayesian_optimization/init_points/{num_init_points}_init_points_emissions.pkl\', "wb") as f:\n     pickle.dump(init_points, f)\n'

To then fit a GP to these points

In [7]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_random_init = GPyModelWrapper(emulator, n_restarts=5)
emukit_model_random_init.optimize()

Optimization restart 1/5, f = 1297.6948600888115
Optimization restart 2/5, f = 1593.0598703683074
Optimization restart 3/5, f = 1201.1270324900768
Optimization restart 4/5, f = 1201.1270324913662
Optimization restart 5/5, f = 1201.1270324895102


### Experimentally designed emulator with model variance

We also load the emulator obtained during experimental design:

In [8]:
emukit_model_variance = pickle.load(open('experimental_design/models/model_variance_20_init_points_250_loops_co2.pkl', 'rb'))

### Experimentally designed emulator with integrated variance reduction

We also load the emulator obtained during experimental design:

In [9]:
emukit_model_integrated_variance = pickle.load(open('experimental_design/models/integrated_variance_reduction_20_init_points_250_loops_co2.pkl', 'rb'))

## Bayesian optimisation

We now run bayesian optimisation on the emulators

In [10]:
n_iter_bo = 50

### Acquisition functions

In [11]:
acquisition_random_init = ExpectedImprovement(emukit_model_random_init)

In [12]:
acquisition_model_variance = ExpectedImprovement(emukit_model_variance)

In [13]:
acquisition_integrated_variance = ExpectedImprovement(emukit_model_integrated_variance)

### Optimisation loop

In [14]:
bo_random_init = BayesianOptimizationLoop(parameter_space, emukit_model_random_init, acquisition=acquisition_random_init)
bo_random_init.run_loop(user_function_emissions, n_iter_bo)

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1201.1270324895102
Optimization restart 2/5, f = 1201.1270324894117
Optimization restart 3/5, f = 1201.1270324899792
Optimization restart 4/5, f = 1201.1270338901209
Optimization restart 5/5, f = 1201.127033322709
[[12. 25.  5. 70.  3.  5.]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 8.876725628276361

Optimization restart 1/5, f = 1206.0713298115704


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 2/5, f = 1206.057500229444
Optimization restart 3/5, f = 1206.0575003020144
Optimization restart 4/5, f = 1206.0575002305839
Optimization restart 5/5, f = 1206.0575002463088
[[12.         25.          5.         44.51789345  1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 13.27183348369983



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


...

Optimization restart 1/5, f = 1397.895835642382
Optimization restart 2/5, f = 1397.895835796307
Optimization restart 3/5, f = 1397.8958356351936
Optimization restart 4/5, f = 1397.8958362567316
Optimization restart 5/5, f = 1397.8958367338923
[[11.         15.41434892  5.         64.4383285   3.          5.        ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 9.572842517240387



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1401.508409759594
Optimization restart 2/5, f = 1401.5084022911296
Optimization restart 3/5, f = 1401.5084022889148


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 4/5, f = 1401.5084022890883
Optimization restart 5/5, f = 1401.5084023151453


In [15]:
bo_model_variance = BayesianOptimizationLoop(parameter_space, emukit_model_variance, acquisition=acquisition_model_variance)
bo_model_variance.run_loop(user_function_emissions, n_iter_bo)

Optimization restart 1/5, f = 1299.81783983801
Optimization restart 2/5, f = 1299.8178398490209
Optimization restart 3/5, f = 1299.8178418079897
Optimization restart 4/5, f = 1342.8603518625468
Optimization restart 5/5, f = 1625.4010799965722
[[20.         25.          5.         64.22771518  1.          5.        ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 5.995269942563651

Optimization restart 1/5, f = 1303.6884926972411


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 2/5, f = 1346.8168688956505
Optimization restart 3/5, f = 1631.025641578025
Optimization restart 4/5, f = 1303.6884552327747
Optimization restart 5/5, f = 1346.8168699981195
[[20.         13.91617844  5.         60.68796114  1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 6.458436413889607

Optimization restart 1/5, f = 1307.5599285554697


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 2/5, f = 1307.5599016343454
Optimization restart 3/5, f = 1636.64861430059
Optimization restart 4/5, f = 1350.2547253901196
Optimization restart 5/5, f = 1307.5599016088222
[[20.         13.91953606  5.         60.74986437  1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 6.362534690326308

Optimization restart 1/5, f = 1311.3302419623085


...

Optimization restart 5/5, f = 1849.0870174915212
[[20.          8.         22.76698636 57.23949169  1.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 13.85876243742351

Optimization restart 1/5, f = 1466.836883156243


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 2/5, f = 1484.1358401754992


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 3/5, f = 1486.713732529585
Optimization restart 4/5, f = 1484.1358400303582
Optimization restart 5/5, f = 1484.1358400296135


In [16]:
bo_integrated_variance = BayesianOptimizationLoop(parameter_space, emukit_model_integrated_variance, acquisition=acquisition_integrated_variance)
bo_integrated_variance.run_loop(user_function_emissions, n_iter_bo)

Optimization restart 1/5, f = 1281.5336113895025
Optimization restart 2/5, f = 1281.5336114425982
Optimization restart 3/5, f = 1281.5336114215602
Optimization restart 4/5, f = 1281.5336113873814
Optimization restart 5/5, f = 1281.5336113860117
[[11.         25.          5.         37.01831831  3.          1.5       ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 18.396061576246655



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1287.1756666936026
Optimization restart 2/5, f = 1287.1625539995025
Optimization restart 3/5, f = 1287.1625539995625
Optimization restart 4/5, f = 1287.162554044373
Optimization restart 5/5, f = 1287.1625539998713
[[15.         23.53393966  5.         44.15457675  3.          5.        ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 12.553583108078845

Optimization restart 1/5, f = 1291.4884044922787


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 2/5, f = 1291.4878580493578
Optimization restart 3/5, f = 1291.4878579546591
Optimization restart 4/5, f = 1291.48785795465
Optimization restart 5/5, f = 1291.4878579549138
[[20.         19.24980206  5.         45.48532926  1.          5.        ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 8.577261880336168

Optimization restart 1/5, f = 1295.6617080050942


...

Optimization restart 1/5, f = 1479.7584474847213
Optimization restart 2/5, f = 1479.7584475288045


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 3/5, f = 1483.7937030358985
Optimization restart 4/5, f = 1479.7584474648404
Optimization restart 5/5, f = 1485.017818163906
[[20.          8.          5.         68.28298044  3.          5.        ]]

User function emissions called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 6.174908944512381



 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1483.5370712035815
Optimization restart 2/5, f = 1483.5370697936955
Optimization restart 3/5, f = 1483.5370697936983
Optimization restart 4/5, f = 1483.5370698023846
Optimization restart 5/5, f = 1483.5370697955525


## Results

In [17]:
parameters_name = ['Size of the grid', 'Speed limit in the net (m/s)', 'Max vehicles speed (m/s)', 'Length of the roads (m)', 'Number of lanes', 'Accelleration (m/s^2)']

### Results on the random initialised emulator

In [18]:
results_random_init = bo_random_init.get_results()

In [19]:
results_random_init_df = pd.DataFrame(results_random_init.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,8.0,5.0,70.0,1.0,1.5


In [20]:
results_random_init.minimum_value

5.469575516984683

### Results on the experimentally designed emulator with model variance

In [21]:
results_model_variance = bo_model_variance.get_results()

In [22]:
results_random_init_df = pd.DataFrame(results_model_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,25.0,5.0,70.0,1.0,1.5


In [23]:
results_model_variance.minimum_value

5.469575516984683

### Results on the experimentally designed emulator with integrated variance reduction

In [24]:
results_integrated_variance = bo_integrated_variance.get_results()

In [25]:
results_random_init_df = pd.DataFrame(results_integrated_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,25.0,5.0,70.0,1.0,1.5


In [26]:
results_integrated_variance.minimum_value

5.469575516984683

## Save stuff

In [27]:
with open(f'bayesian_optimization/bayesian_opt_results/results_random_init_emissions.pkl', "wb") as f:
     pickle.dump(results_random_init, f)

with open(f'bayesian_optimization/bayesian_opt_results/results_model_variance_emissions.pkl', "wb") as f:
     pickle.dump(results_model_variance, f)
        
with open(f'bayesian_optimization/bayesian_opt_results/results_integrated_variance_emissions.pkl', "wb") as f:
     pickle.dump(results_integrated_variance, f)

In [28]:
with open(f'bayesian_optimization/models/270_random_init_50_bay_opt_emissions.pkl', "wb") as f:
     pickle.dump(emukit_model_random_init, f)

with open(f'bayesian_optimization/models/270_model_variance_50_bay_emissions.pkl', "wb") as f:
     pickle.dump(emukit_model_variance, f)
        
with open(f'bayesian_optimization/models/270_integrated_variance_50_bay_emissions.pkl', "wb") as f:
     pickle.dump(emukit_model_integrated_variance, f)